In [20]:
using MaxEntChemostat2018;
using DataFrames;

In [2]:
S, mets, rxns = MaxEntChemostat2018.FBA.load_cho();

## Tools

In [4]:
model_formats = [:simple, :cossio, :extended];
function check_keys(df, ks)
    dfks = names(df);
    for k in ks
        if findfirst(dfks,k) == 0
            error(":$(string(k)) key is missing!!!") 
        end
    end
end

check_keys (generic function with 1 method)

## S

In [5]:
function check_S(S)
    if isempty(S) error("S is empty!!") end;
    if !all(typeof.(S) .== Float64) error("S must contain Float64 elements.") end;
    if all(S .== 0.0) error("All elements in S are zeros.") end;
end

check_S (generic function with 1 method)

### Simple format

In [34]:
simple_format_keys = Dict("mets" => [:id], "rxns" => [:id,:lb,:ub]);

function check_simple_compat(S, mets, rxns)
    # S
    check_S(S);
    
    # Mets
    # keys
    check_keys(mets, simple_format_keys["mets"]);
    # Types
    if eltype(mets[:id]) != String error("mets :ids must be Strings!!!"); end
    # Data quality
    if !allunique(mets[:id]) error("mets contains repeated ids!!!"); end

    # Rxns
    # keys
    check_keys(rxns, simple_format_keys["rxns"]);
    # Types
    if eltype(rxns[:id]) != String error("rxns :ids must be Strings!!!"); end
    if eltype(rxns[:ub]) != Float64 error("rxns :ub must contains Float64 elements!!!"); end
    if eltype(rxns[:lb]) != Float64 error("rxns :lb must contains Float64 elements!!!"); end
    # Data quality
    if !allunique(rxns[:id]) error("rxns contains repeated ids!!!"); end
    if !all(rxns[:ub] .>= 0.0) error("rxns :ub values must be positive!!!"); end
    if !all(rxns[:lb] .<= 0.0) error("rxns :lb values must be negative!!!"); end 
    
end

check_simple_compat (generic function with 1 method)

### Cossio format

In [13]:
cossio_format_keys = Dict("mets" => [:id, :L, :V, :e, :y, :c], "rxns" => [:id, :ub, :lb, :ap, :an]);

function check_cossio_compat(S, mets, rxns)
    # S
    check_S(S);
    
    # Mets
    # keys
    check_keys(mets, cossio_format_keys["mets"])
    # Types
    if eltype(mets[:id]) != String error("mets :ids must be Strings!!!"); end
    if eltype(mets[:L]) != Float64 error("mets :pV must contains Float64 elements!!!"); end
    if eltype(mets[:V]) != Float64 error("mets :pL must contains Float64 elements!!!"); end
    if eltype(mets[:e]) != Float64 error("mets :e must contains Float64 elements!!!"); end
    if eltype(mets[:y]) != Float64 error("mets :y must contains Float64 elements!!!"); end
    # Data quality
    if !allunique(mets[:id]) error("mets contains repeated ids!!!"); end
    if !all(mets[:V] .>= 0.0) error("mets :V values must be positive!!!"); end
    if !all(mets[:L] .<= 0.0) error("mets :L values must be negative!!!"); end
    if !all(mets[:V] .>= mets[:L]) error("mets :V must be greater than :L!!!"); end
    
    # Rxns
    # keys
    check_keys(rxns, cossio_format_keys["rxns"]);
    # Types
    if eltype(rxns[:id]) != String error("rxns :ids must be Strings!!!"); end
    if eltype(rxns[:ub]) != Float64 error("rxns :ub must contains Float64 elements!!!"); end
    if eltype(rxns[:lb]) != Float64 error("rxns :lb must contains Float64 elements!!!"); end
    if eltype(rxns[:ap]) != Float64 error("rxns :ap must contains Float64 elements!!!"); end
    if eltype(rxns[:an]) != Float64 error("rxns :an must contains Float64 elements!!!"); end
    # Data quality
    if !allunique(rxns[:id]) error("rxns contains repeated ids!!!"); end
    if !all(rxns[:ub] .>= 0.0) error("rxns :ub values must be positive!!!"); end
    if !all(rxns[:lb] .<= 0.0) error("rxns :lb values must be negative!!!"); end
    if !all(rxns[:ap] .>= 0.0) error("rxns :ap values must be positive!!!"); end
    if !all(rxns[:an] .>= 0.0) error("rxns :an values must be positive!!!"); end
end

check_cossio_compat (generic function with 2 methods)

### Extended Format

In [17]:
extended_format_keys = Dict("mets" => [:id, :pV, :pL, :nV, :nL, :e, :y, :c],
                                "rxns" => [:id, :pub, :plb, :nub, :nlb, :ap, :an]);

"""
    Test the data to the extended format!!!
    See extended_format_contract for more info.
"""
function check_extended_compat(S, mets, rxns)
    #S
    check_S(S);
    
    #Mets
    # keys
    check_keys(mets, extended_format_keys["mets"]);
    # Types
    if eltype(mets[:id]) != String error("mets :ids must be Strings!!!"); end
    if eltype(mets[:pV]) != Float64 error("mets :pV must contains Float64 elements!!!"); end
    if eltype(mets[:pL]) != Float64 error("mets :pL must contains Float64 elements!!!"); end
    if eltype(mets[:nV]) != Float64 error("mets :nV must contains Float64 elements!!!"); end
    if eltype(mets[:nL]) != Float64 error("mets :nL must contains Float64 elements!!!"); end
    if eltype(mets[:e]) != Float64 error("mets :e must contains Float64 elements!!!"); end
    if eltype(mets[:y]) != Float64 error("mets :y must contains Float64 elements!!!"); end
    # Data quality
    if !allunique(mets[:id]) error("mets contains repeated ids!!!"); end
    if !all(mets[:pV] .>= 0.0) error("mets :pV values must be positive!!!"); end
    if !all(mets[:pL] .>= 0.0) error("mets :pL values must be positive!!!"); end
    if !all(mets[:pV] .>= mets[:pL]) error("mets :pV must be more positive than :pL!!!"); end
    if !all(mets[:nV] .<= 0.0) error("mets :nV values must be negative!!!"); end
    if !all(mets[:nL] .<= 0.0) error("mets :nL values must be negative!!!"); end
    if !all(mets[:nV] .<= mets[:nL]) error("mets :pV must be more negative than :pL!!!"); end
    
    # Rxns
    # keys
    check_keys(rxns, extended_format_keys["rxns"]);
    # Types
    if eltype(rxns[:id]) != String error("rxns :ids must be Strings!!!"); end
    if eltype(rxns[:pub]) != Float64 error("rxns :pub must contains Float64 elements!!!"); end
    if eltype(rxns[:plb]) != Float64 error("rxns :plb must contains Float64 elements!!!"); end
    if eltype(rxns[:nub]) != Float64 error("rxns :nub must contains Float64 elements!!!"); end
    if eltype(rxns[:nlb]) != Float64 error("rxns :nlb must contains Float64 elements!!!"); end
    if eltype(rxns[:ap]) != Float64 error("rxns :ap must contains Float64 elements!!!"); end
    if eltype(rxns[:an]) != Float64 error("rxns :an must contains Float64 elements!!!"); end
    # Data quality
    if !allunique(rxns[:id]) error("rxns contains repeated ids!!!"); end
    if !all(rxns[:pub] .>= 0.0) error("rxns :pub values must be positive!!!"); end
    if !all(rxns[:plb] .>= 0.0) error("rxns :plb values must be positive!!!"); end
    if !all(rxns[:pub] .>= rxns[:pub]) error("rxns :pub must be more positive or equal than :pub!!!"); end
    if !all(rxns[:nub] .<= 0.0) error("rxns :nub values must be negative!!!"); end
    if !all(rxns[:nlb] .<= 0.0) error("rxns :nlb values must be negative!!!"); end
    if !all(rxns[:nub] .<= rxns[:nlb]) error("rxns :nub must be more negative or equal than :nlb!!!"); end
    if !all(rxns[:ap] .>= 0.0) error("rxns :ap values must be positive!!!"); end
    if !all(rxns[:an] .>= 0.0) error("rxns :an values must be positive!!!"); end
end


check_extended_compat

In [15]:
function get_compat_format(S, mets, rxns)
    try check_extended_compat(S, mets, rxns); return :extended
        catch try check_cossio_compat(S,mets, rxns); return :cossio
            catch try check_simple_compat(S,mets,rxns); return :simple
                catch error("The data is not compatible with any format!!!" * 
                   " Use individual format checkers for more info");
            end
        end
    end
end

get_compat_format (generic function with 1 method)

## Converters

In [70]:
function convert_to_simple(model)
    if model.format == :simple
        return model;
    elseif model.format == :cossio
        warn("This convertion usually ends with a broken model," * 
            " it is recomended to check the exchanges, the biomass reaction and the biomass demand")
        # Mets
        simple_mets = DataFrame();
        simple_mets[:id] = model.mets[:id];
        #Rxns
        simple_rxns = DataFrame();
        simple_rxns[:id] = model.rxns[:id];
        simple_rxns[:ub] = model.rxns[:ub];
        simple_rxns[:lb] = model.rxns[:lb];    
        return Model(S, simple_mets, simple_rxns);
        
    elseif model.format == :extended
        warn("This convertion usually ends with a broken model," * 
            " it is recomended to check the exchanges, the biomass reaction and the biomass demand")
        # Mets
        simple_mets = DataFrame();
        simple_mets[:id] = model.mets[:id];
        #Rxns
        simple_rxns = DataFrame();
        simple_rxns[:id] = model.rxns[:id];
        simple_rxns[:ub] = model.rxns[:pub];
        simple_rxns[:lb] = model.rxns[:nub];    
        return Model(S, simple_mets, simple_rxns);
    else
        error("Format $(string(model.format)) not supported yet!!!")
    end
end
convert_to_simple(S,mets,rxns) = convert_to_simple(Model(S, mets, rxns));

function convert_to_cossio(model)
    
    if model.format == :cossio 
        return model; 
    elseif model.format == :simple
        error("A :simple model can't be converted to a :cossio model, data insufficient!!!")
    elseif model.format == :extended
        # Mets
        cossio_mets = DataFrame();
        cossio_mets[:id] = model.mets[:id];
        cossio_mets[:V] = model.mets[:pV];
        cossio_mets[:L] = model.mets[:nV];
        cossio_mets[:e] = model.mets[:e];
        cossio_mets[:y] = model.mets[:y];
        cossio_mets[:c] = model.mets[:c];
        # Rxns
        cossio_rxns = DataFrame();
        cossio_rxns[:id] = model.rxns[:id];
        cossio_rxns[:ub] = model.rxns[:pub];
        cossio_rxns[:lb] = model.rxns[:nub];
        cossio_rxns[:ap] = model.rxns[:ap];
        cossio_rxns[:an] = model.rxns[:an];
        
        return Model(S, cossio_mets, cossio_rxns);
        
    else
        error("Format $(string(model.format)) not supported yet!!!")
    end
    
end
convert_to_cossio(S,mets,rxns) = convert_to_cossio(Model(S,mets,rxns));

function convert_to_extended(model)
    
    if model.format == :extended 
        return model; 
    elseif model.format == :simple
        error("A :simple model can't be converted to a :extended model, data insufficient!!!")
    elseif model.format == :cossio
        
        m,n = size(model.S);
        
        # Mets
        extebnded_mets = DataFrame();
        extebnded_mets[:id] = model.mets[:id];
        extebnded_mets[:pV] = model.mets[:V];
        extebnded_mets[:pL] = zeros(m);
        extebnded_mets[:nV] = model.mets[:L];
        extebnded_mets[:nL] = zeros(m);
        extebnded_mets[:e] = model.mets[:e];
        extebnded_mets[:y] = model.mets[:y];
        extebnded_mets[:c] = model.mets[:c];
        # Rxns
        extebnded_rxns = DataFrame();
        extebnded_rxns[:id] = model.rxns[:id];
        extebnded_rxns[:pub] = model.rxns[:ub];
        extebnded_rxns[:plb] = zeros(n);
        extebnded_rxns[:nub] = model.rxns[:lb];
        extebnded_rxns[:nlb] = zeros(n);
        extebnded_rxns[:ap] = model.rxns[:ap];
        extebnded_rxns[:an] = model.rxns[:an];
        
        return Model(S, extebnded_mets, extebnded_rxns);
        
    else
        error("Format $(string(model.format)) not supported yet!!!")
    end
    
end
convert_to_extended(S, mets, rxns) = convert_to_extended(Model(S, mets, rxns));

In [18]:
struct Model
    S # The Stoichimetric matrix
    mets # Information linked to the metabolites of the net
    rxns # Information linked to the reactions of the net
    format # The format of the data
    Model(S, mets, rxns) = new(S,mets,rxns, get_compat_format(S, mets, rxns));
end

In [60]:
"""
    Just a wrapper of the network data, to linke it to a given format!!!
"""
function Model(S, mets, rxns, format)
    if format == :simple
        convert_to_simple(S, mets, rxns);
    elseif format == :cossio
        convert_to_cossio(S,mets,rxns);
    elseif format == :extended
        convert_to_extended(S, mets, rxns);
    end
end

Model